In [1]:
import numpy as np

def householder_reflection(v):
    """
    Compute the Householder reflection vector.
    """
    I = np.eye(len(v))
    v = v / np.linalg.norm(v)
    return I - 2 * np.outer(v, v)

def householder_tridiagonalize(A):
    """
    Reduce symmetric matrix A to tridiagonal form using Householder's method.
    """
    n = A.shape[0]
    T = A.copy()
    
    for k in range(n - 2):
        x = T[k + 1:, k]
        e = np.zeros_like(x)
        e[0] = np.linalg.norm(x)
        
        u = x - e
        if np.linalg.norm(u) != 0:
            u /= np.linalg.norm(u)
        
        Hk = np.eye(n)
        Hk[k + 1:, k + 1:] -= 2 * np.outer(u, u)
        
        T = Hk @ T @ Hk.T
    
    return T

def sturm_sequence(A, x):
    """
    Calculate the Sturm sequence for tridiagonal matrix A at value x.
    """
    n = A.shape[0]
    p = np.zeros(n + 1)
    p[0] = 1
    p[1] = A[0, 0] - x

    for i in range(2, n + 1):
        p[i] = (A[i - 1, i - 1] - x) * p[i - 1] - A[i - 1, i - 2] ** 2 * p[i - 2]

    return p

def count_sturm_sign_changes(p):
    """
    Count the number of sign changes in the Sturm sequence.
    """
    sign_changes = 0
    for i in range(1, len(p)):
        if p[i] * p[i - 1] < 0:
            sign_changes += 1
    return sign_changes

def find_eigenvalues(A, tol=1e-10, max_iter=1000):
    """
    Find the eigenvalues of symmetric matrix A using Householder's method and Sturm sequence.
    """
    n = A.shape[0]
    T = householder_tridiagonalize(A)

    eigenvalues = []

    for i in range(n):
        lower_bound = np.min(np.diag(T)) - tol
        upper_bound = np.max(np.diag(T)) + tol

        while upper_bound - lower_bound > tol:
            midpoint = (lower_bound + upper_bound) / 2
            sturm_seq = sturm_sequence(T, midpoint)
            num_sign_changes = count_sturm_sign_changes(sturm_seq)

            if num_sign_changes > i:
                upper_bound = midpoint
            else:
                lower_bound = midpoint

        eigenvalues.append((lower_bound + upper_bound) / 2)

    return np.array(eigenvalues)

# Example usage:
A = np.array([[4, 1, 2],
              [1, 2, 3],
              [2, 3, 1]], dtype=float)
eigenvalues = find_eigenvalues(A)
print("Eigenvalues:", eigenvalues)


Eigenvalues: [-0.6         2.30838907  4.        ]
